ba2f -> Implement RandomizedMotifSearch

In [2]:
import numpy as np
from random import randint

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers

def CountMatrix(motifs): 
  count = np.zeros((4,len(motifs[0])))
  for motif in motifs:
    for i in range(len(motif)):
      if motif[i-1] == 'A':
        count[0][i-1] += 1
      elif motif[i-1] == 'C':
        count[1][i-1] += 1
      elif motif[i-1] == 'G':
        count[2][i-1] += 1
      elif motif[i-1] == 'T':
        count[3][i-1] += 1
  return count

def ProfileMatrix(motifs):
  countMat = CountMatrix(motifs)
  pseudoMat = PseudoCountMatrix(countMat)
  profileMatrx = np.zeros((4,len(motifs[0])))
  for row in range(4):
    for col in range(k):
      profileMatrx[row][col] = pseudoMat[row][col] / len(motifs)
  return profileMatrx

def PseudoCountMatrix(countMat):
  pseudo = [1*4]*len(countMat[0])
  pseudoMatrix = pseudo+countMat
  return pseudoMatrix


def ProfileMostProbableKmer(genome, k, profile_matrix):
  maxprob = 0
  kmer = genome[0:k]
  for i in range(1,len(genome) - k +1):
    prob = 1
    pattern = genome[i:i+k]
    for j in range(k):
      l = symbolToNumber(pattern[j])
      prob *= profile_matrix[l][j]
    if prob > maxprob:
      maxprob = prob
      kmer = pattern
  return kmer
  
def symbolToNumber(symbol):
	if symbol == "A":
		return 0
	if symbol == "C":
		return 1
	if symbol == "G":
		return 2
	if symbol == "T":
		return 3

def NumberToSymbol(number):
	if number == 0:
		return "A"
	if number == 1:
		return "C"
	if number == 2:
		return "G"
	if number == 3:
		return "T"

def Consensus(profile):
  consensus = ""
  for col in range(len(profile[0])):  
    max = 0
    loc = 0
    for row in range(4):
      if profile[row][col] > max:
        loc = row  
        max = profile[row][col]  
    consensus += NumberToSymbol(loc)  
  return consensus

neucleotide_base = {0:'A', 1:'C', 2:'G', 3:'T'}
def NumberToPattern(index, k):
    if(k == 1): #base case
        return neucleotide_base[index]
    pref_index = int(index/4)
    rem = int(index%4)
    symbol = neucleotide_base[rem] 
    pref_pattern = numberToPattern(pref_index, k-1) 
    
    return pref_pattern+symbol

def Score(motifs):
  profile = ProfileMatrix(motifs)
  cons = Consensus(profile)
  score = 0
  for motif in motifs: 
    for i in range(len(motif)): 
      if cons[i] != motif[i]:
        score+=1
  return score

def getRandom(n):
  return randint(0, n)

def FormRandomlyMotifs(genomes, k):
  motifs = []
  for genome in genomes:
    start = getRandom(len(genome)-k)
    motifs.append(genome[start:start+k])
  return motifs

def generateMotifsSet(profile_matrix, k, genomes):
  probKmers = []
  for genome in genomes:
    probKmers.append(ProfileMostProbableKmer(genome, k, profile_matrix))
  return probKmers

def RandomizedMotifSearch(genomes, k, t):
  motifs = FormRandomlyMotifs(genomes, k)
  bestMotifs = motifs
  while True:
    profile_matrix = ProfileMatrix(motifs)
    motifs = generateMotifsSet(profile_matrix, k, genomes)
    if Score(motifs) < Score(bestMotifs):
      bestMotifs = motifs
    else:
      return bestMotifs


k = int(input("Enter pattern length: "))
t = int(input("Enter no. of Genomes: "))
with open('/content/rosalind_ba2f.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]
  
best_score = float("Inf")
best_result = []
for i in range(1000):
  ans = RandomizedMotifSearch(genomelist, k, t)
  curr_score = Score(ans)
  if curr_score <= best_score:
    best_score = curr_score
    best_result = ans
for line in best_result:
  print(line)

Enter pattern length: 8
Enter no. of Genomes: 5
TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG


ba2g -> Implement GibbsSampler

In [7]:
import numpy as np
import random
from random import randint, choices

def FormBestMotifs(genomes, k,t):
  bmotifs = []
  for genome in genomes:
    bmotifs.append(genome[:k])
  return bmotifs

def generateTextKmer(text, k):
  kmers = []
  for i in range(len(text)-k+1):
    kmers.append(text[i:i+k])
  return kmers


def FormCountMatrix(motifs, k): 
  count = np.zeros((4,len(motifs[0])))
  for motif in motifs:
    for i in range(len(motif)):
      if motif[i-1] == 'A':
        count[0][i-1] += 1
      elif motif[i-1] == 'C':
        count[1][i-1] += 1
      elif motif[i-1] == 'G':
        count[2][i-1] += 1
      elif motif[i-1] == 'T':
        count[3][i-1] += 1
  return count

def FormProfile(motifs, k): 
  countMat = FormCountMatrix(motifs, k)
  pseudoMat = PseudoCountMatrix(countMat, k)
  profileMatrx = np.zeros((4,len(motifs[0])))
  for row in range(4):
    for col in range(k):
      profileMatrx[row][col] = pseudoMat[row][col] / len(motifs)
  return profileMatrx

def PseudoCountMatrix(countMat, k): 
  pseudo = [1*4] * k
  pseudoMatrix = pseudo + countMat
  return pseudoMatrix

  
def symbolToNumber(symbol):
	if symbol == "A":
		return 0
	if symbol == "C":
		return 1
	if symbol == "G":
		return 2
	if symbol == "T":
		return 3

def NumberToSymbol(number):
	if number == 0:
		return "A"
	if number == 1:
		return "C"
	if number == 2:
		return "G"
	if number == 3:
		return "T"

def Consensus(profile):
  consensus = ""
  for col in range(len(profile[0])):  
    max = 0
    loc = 0
    for row in range(4):
      if profile[row][col] > max:
        loc = row  
        max = profile[row][col]
    consensus += NumberToSymbol(loc)  
  return consensus


def Score(motifs, k):
  profile = FormProfile(motifs, k)
  cons = Consensus(profile)
  score = 0
  for motif in motifs:  
    for i in range(len(motif)): 
      if cons[i] != motif[i]:
        score+=1
  return score

def getRandom(n):
  return randint(0, n)

def FormRandomlyMotifs(genomes, k):
  motifs = []
  for genome in genomes:
    start = getRandom(len(genome)-k)
    motifs.append(genome[start:start+k])
  return motifs

def generateMotifsSet(profile_matrix, k, genomes):
  probKmers = []
  for genome in genomes:
    probKmers.append(ProfileMostProbableKmer(genome, k, profile_matrix))
  return probKmers

def probability(kmer, profile_matrix):
  prob = 1
  for j in range(len(kmer)):
      l = symbolToNumber(kmer[j])
      prob *= profile_matrix[l][j]
  return prob

def ProfileRandomGeneratedKmer(profile_matrix, k, genome):
  prob = []
  kmers = generateTextKmer(genome, k)
  for kmer in kmers:
    prob.append(probability(kmer, profile_matrix))
  
  motifs = random.choices(kmers, prob)
  return motifs[0]


def GibbsSampler(genomes, k, t, N):
  motifs = FormRandomlyMotifs(genomes, k)
  bestMotifs = motifs
  for j in range(N):
    i = getRandom(t-1)
    profile_matrix = FormProfile(motifs[:i] + motifs[i+1 :], k)
    motifi = ProfileRandomGeneratedKmer(profile_matrix, k, genomes[i])
    motifs[i] = motifi 
    if Score(motifs, k) < Score(bestMotifs, k):
      bestMotifs = motifs 
  return bestMotifs

k = 8
t = 5  
N = 100 
with open('/content/rosalind_ba2g.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]

best = GibbsSampler(genomelist, k, t, N)
best_score = Score(best, k)
best_result = []
for i in range(20):
  ans = GibbsSampler(genomelist, k, t, N)
  curr_score = Score(ans, k)
  if curr_score < best_score:
    best_score = curr_score
    best_result = ans
for line in best_result:
    print(line)

CAGTAAAC
GTGTAAGT
CCGAAAGA
GTTTCAGG
ATGTTGGC


ba3a -> Generate the k-mer Composition of a String

In [ ]:
def KmerCompositionofString(genome, k):
  kmers = []
  for i in range(len(genome)-k+1):
    kmers.append(genome[i:i+k])
  kmers.sort()
  return kmers

k = int(input("Enter the length of pattern"))
genome = input("Enter the string: ")
ans = KmerCompositionofString(genome, k)
#print("The answer is: ")
print(*ans, sep='\n')

Enter the length of pattern5
Enter the string: CAATCCAAC
AATCC
ATCCA
CAATC
CCAAC
TCCAA


ba3b -> Reconstruct a String from its Genome Path


In [ ]:
def ReconstructGenomePath(genomelist):
  path = genomelist[0] #start genome
  print(path)
  for i in range(1, len(genomelist)):
    path += genomelist[i][-1] #[-1] returns the last charachter of a string
  return path

genomelist = []
#read a file line by line into a list, (readling file line into genomelist)
# with open(filename) as f:
#     content = f.readlines()
with open('/content/ReconstructString.txt', 'r') as f:
        genomelist = [line.strip() for line in f.readlines()] #content = [x.strip() for x in content] -> removes whitespace charachters ('\n) from end of each line
ans = ReconstructGenomePath(genomelist)
print(ans)

    

ACCGA
ACCGAAGCT


ba3c -> Construct the Overlap Graph of a Collection of k-mers

In [ ]:
def OverlapGraph(genomelist):
  genomelist.sort()
  for genome in genomelist:
    for gene in genomelist:
      if SuffixofString(genome) == PrefixofString(gene):
        print(genome +" -> "+gene)

def SuffixofString(string):
    return string[1:len(string)]

def PrefixofString(string):
    return string[0:len(string)-1]

genomelist = []
with open('/content/OverlapGraphStrings.txt', 'r') as f:
        genomelist = [line.strip() for line in f.readlines()] 
OverlapGraph(genomelist)

AGGCA -> GGCAT
CATGC -> ATGCG
GCATG -> CATGC
GGCAT -> GCATG


ba3d -> Construct the De Bruijn Graph of a String

In [ ]:
def KmerCompositionofString(genome, k):
  kmers = []
  for i in range(len(genome)-k+1):
    kmers.append(genome[i:i+k])
  kmers.sort()
  return kmers 

def SuffixofString(string):
    return string[1:len(string)]

def PrefixofString(string):
    return string[0:len(string)-1]

def DeBruign(k, genome):
  result={}
  kmers = KmerCompositionofString(genome, k)
  for kmer in kmers:
    if PrefixofString(kmer) not in result:
      result[PrefixofString(kmer)] = SuffixofString(kmer)  #AAGA if mydict[AAG] is not in diction
    else:
      result[PrefixofString(kmer)] += ',' + SuffixofString(kmer)
  return result

k = int(input("Enter the length of kmer: "))
genome = input("Enter the genome: ")
ans = DeBruign(k, genome)
for key, value in  sorted(ans.items()): #sort dictionary by keys
  print(key +" -> "+ value)



Enter the length of kmer: 4
Enter the genome: AAGATTCTCTAC
AAG -> AGA
AGA -> GAT
ATT -> TTC
CTA -> TAC
CTC -> TCT
GAT -> ATT
TCT -> CTA,CTC
TTC -> TCT


ba3e -> Construct the De Bruijn Graph of a Collection of k-mers

In [ ]:
def SuffixofString(string):
    return string[1:len(string)]

def PrefixofString(string):
    return string[0:len(string)-1]

def DeBruign(kmers):
  result={}
  for kmer in kmers:
    if PrefixofString(kmer) not in result:
      result[PrefixofString(kmer)] = SuffixofString(kmer)  #AAGA if mydict[AAG] is not in diction
    else:
      result[PrefixofString(kmer)] += ',' + SuffixofString(kmer)
  return result

kmers = []
with open('/content/DeBruignFromKmer.txt', 'r') as f:
        kmers = [line.strip() for line in f.readlines()] 
ans = DeBruign(kmers)
for key, value in  sorted(ans.items()): #sort dictionary by keys
  print(key +" -> "+ value)

AGG -> GGG
CAG -> AGG,AGG
GAG -> AGG
GGA -> GAG
GGG -> GGG,GGA
